# Executing Gradient-based Evasion Attacks

The security of models can be evaluated through the creation of *adversarial EXEmples*, carefully-manipulated Windows programs that evade detection of AI-based detectors.
These can be formalized as a maximization problem as follows:
$$ \max_\delta L(h(x;\delta), y) $$
where $\delta$ is the manipulation to compute in order to maximize the classification error of the model.

In the worst-case scenario is posed by attackers with access to the target model. If this is also differentiable, they can compute *gradient-based*, since the gradient provides the best direction to follow to maximise the output of the loss function.

We start by loading end-to-end models which can be attacked in this way, since they are differentiable and provide predictions without using any non-invertible feature extractor.

In [1]:
%%capture --no-stderr
try:
    import maltorch
except ImportError:
   %pip install git+https://github.com/zangobot/maltorch

In [2]:
from pathlib import Path
from maltorch.data.loader import load_from_folder, create_labels
from maltorch.data_processing.grayscale_preprocessing import GrayscalePreprocessing
from maltorch.zoo.avaststyleconv import AvastStyleConv
from maltorch.zoo.bbdnn import BBDnn
from maltorch.zoo.malconv import MalConv
from maltorch.zoo.resnet18 import ResNet18
from secmlt.metrics.classification import Accuracy
from torch.utils.data import DataLoader, TensorDataset

exe_folder = Path("insert_here_path")
device = "cpu"
networks = {
    'BBDnn': BBDnn.create_model(device=device),
    'Malconv': MalConv.create_model(device=device),
    'AvastStyleConv': AvastStyleConv.create_model(device=device),
    'Grayscale ResNet18': ResNet18.create_model(
        preprocessing=GrayscalePreprocessing(),
        device=device),
}

X = load_from_folder(exe_folder, "exe", device=device)
y = create_labels(X, 1)
data_loader = DataLoader(TensorDataset(X, y), batch_size=3)

RuntimeError: received an empty list of sequences

After having loaded models, we can now instantiate attacks!
In `maltorch` this is pretty straight-forward, since you just need to create an object and invoke it as a function.
The attack leverage the optimizer named Byte Gradient Descent\citet{demetrio2019explaining}, which substitutes selected bytes with ones that decrease the loss function.

In [ ]:
from maltorch.adv.evasion.content_shift import ContentShift

# This attack expands the gap between the Optional Header and the first section.

grad_attack = ContentShift(
    query_budget=10,
    device=device,
    perturbation_size=2048,
    step_size=256
)

for k in networks:
    print(k)
    model = networks[k]
    adv_dl = grad_attack(model, data_loader)
    print("- - - Pre-attack accuracy: ", Accuracy()(model, data_loader))
    print("- - - Accuracy: ", Accuracy()(model, adv_dl))

That's it! You can swap attacks with ease, by just instantiating them accoredingly.